In [ ]:
pip install gradio

In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
file_path = "https://raw.githubusercontent.com/Premalatha-success/Datasets/refs/heads/main/Churn_Modelling.csv"
df = pd.read_csv(file_path)

# Drop irrelevant columns
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

# Encode categorical variables
le_gender = LabelEncoder()
df["Gender"] = le_gender.fit_transform(df["Gender"])

le_geo = LabelEncoder()
df["Geography"] = le_geo.fit_transform(df["Geography"])

# Define features and target
X = df.drop("Exited", axis=1)
y = df["Exited"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Fix unknown values in prediction function
def predict_churn(CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts,
                  HasCrCard, IsActiveMember, EstimatedSalary):
    try:
        # Handle unknown Geography values
        if Geography not in le_geo.classes_:
            return "Invalid Geography Input"

        # Handle unknown Gender values
        if Gender not in le_gender.classes_:
            return "Invalid Gender Input"

        # Encode categorical inputs
        Geography = le_geo.transform([Geography])[0]
        Gender = le_gender.transform([Gender])[0]

        # Prepare input array
        input_data = np.array([[CreditScore, Geography, Gender, Age, Tenure, Balance,
                                NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary]])

        # Scale input
        input_data = scaler.transform(input_data)

        # Predict
        prediction = model.predict(input_data)[0]
        return "Churn" if prediction == 1 else "Not Churn"

    except Exception as e:
        return f"Error: {str(e)}"

# Deploy with Gradio
inputs = [
    gr.Number(label="Credit Score"),
    gr.Dropdown(le_geo.classes_.tolist(), label="Geography"),  # Ensure dropdown values match trained labels
    gr.Dropdown(le_gender.classes_.tolist(), label="Gender"),  # Ensure dropdown values match trained labels
    gr.Number(label="Age"),
    gr.Number(label="Tenure"),
    gr.Number(label="Balance"),
    gr.Number(label="Number of Products"),
    gr.Radio([0, 1], label="Has Credit Card"),
    gr.Radio([0, 1], label="Is Active Member"),
    gr.Number(label="Estimated Salary")
]

output = gr.Textbox(label="Churn Prediction")

gr.Interface(fn=predict_churn, inputs=inputs, outputs=output, title="Churn Prediction Model").launch()
